### Instance analysis
In this notebook, we just want to analyze the instances of the dataset

In [1]:
import os
from tqdm import tqdm
import pandas as pd

from trigger_arc_tsp.gurobi_tsp_model import GurobiTSPModel
from trigger_arc_tsp.instance import Instance
from trigger_arc_tsp.utils import INSTANCES_DIR

In [2]:
# list instances in the instances directory
instance_names = (os.listdir(os.path.join(INSTANCES_DIR, "instances_release_1")))

# sort according to the instance number. This is grf{num}.txt
instance_names.sort(key=lambda x: int(x[3:-4]))

# load all instances
instances = []
for instance_name in tqdm(instance_names):
    instance = Instance.load_instance_from_file(os.path.join(INSTANCES_DIR, "instances_release_1", instance_name))
    instances.append(instance)

  0%|          | 0/21 [00:00<?, ?it/s]

100%|██████████| 21/21 [00:20<00:00,  1.02it/s]


In [3]:
# Print the N, A, and R values for each instance
res = pd.DataFrame({
    "name": [instance.name.split("/")[1] for instance in instances],
    "N": [instance.N for instance in instances],
    "A": [instance.A for instance in instances],
    "R": [instance.R for instance in instances]
})

print(res, "\n")

# Counts of N
print(res["N"].value_counts())



         name   N     A        R
0    grf1.txt  20   300     5651
1    grf2.txt  20   300     5708
2    grf3.txt  20   300    55306
3    grf4.txt  20   300    58208
4    grf5.txt  20   200     1880
5    grf6.txt  20   200     1732
6    grf7.txt  40  1200    47777
7    grf8.txt  40  1200    47281
8    grf9.txt  40  1200   909054
9   grf10.txt  40  1200   881876
10  grf11.txt  40   800    15490
11  grf12.txt  40   800    15650
12  grf13.txt  60  2700   162514
13  grf14.txt  60  2700   160095
14  grf15.txt  60  2700  4527944
15  grf16.txt  60  2700  4478927
16  grf17.txt  60  1800    53313
17  grf18.txt  60  1800    53472
18  grf19.txt  20   300     5708
19  grf20.txt  40  1200    47281
20  grf21.txt  60  2700   160095 

N
20    7
40    7
60    7
Name: count, dtype: int64


In [7]:

# Check how fast do we solve the TSP for each instance and add it to a column called "TSP_time"
def add_tsp_time():
    if "TSP_time" in res.columns:
        return
    res["TSP_time"] = 0.0
    for i, instance in enumerate(instances):
        model = GurobiTSPModel(instance)
        model.formulate()
        model.solve_to_optimality()

        tour = model.get_tour()
        obj = instance.compute_objective(tour)
        instance.save_solution(tour, obj)
        
        res.iloc[i, res.columns.get_loc("TSP_time")] = model.get_model().Runtime

add_tsp_time()
print(res)

         name   N     A        R   TSP_time
0    grf1.txt  20   300     5651   0.349258
1    grf2.txt  20   300     5708   0.353478
2    grf3.txt  20   300    55306   0.293293
3    grf4.txt  20   300    58208   0.633875
4    grf5.txt  20   200     1880   0.031983
5    grf6.txt  20   200     1732   0.065688
6    grf7.txt  40  1200    47777   0.743205
7    grf8.txt  40  1200    47281   2.674696
8    grf9.txt  40  1200   909054   0.830640
9   grf10.txt  40  1200   881876   6.397891
10  grf11.txt  40   800    15490   0.163518
11  grf12.txt  40   800    15650   1.811374
12  grf13.txt  60  2700   162514   1.171736
13  grf14.txt  60  2700   160095  87.285900
14  grf15.txt  60  2700  4527944   4.346083
15  grf16.txt  60  2700  4478927  24.728470
16  grf17.txt  60  1800    53313   0.212033
17  grf18.txt  60  1800    53472  25.621989
18  grf19.txt  20   300     5708   0.221357
19  grf20.txt  40  1200    47281   1.341986
20  grf21.txt  60  2700   160095  94.326014
